7 Evaluation measures
<br>
7.1 Dice score
<br>
The dice score is a method to determine the accuracy of the segmentation. Thereby, a ground truth image is compared with
the result image of the algorithm. The dice score is defined as follows:
<br>
$$ DSC = \frac{ 2 \times |S \cap G|}{|S| + |G|} $$
<br>
The intersection is the number of pixels that have the same value in the ground truth and result image. The doubled
intersection is divided by the total number of pixels in the ground truth and result image. This delivers a number
between zero and one. The higher the value the better was the performance of the algorithm.
<br>
Another implementation of the dice score would be weighting equally between background and cell pixels. This makes it possible
to consider even small regions, for example when there are proportionally few cells compared to the background.

$$\overline{DSC}=\frac{1}{2} \sum\limits_{i=1}^{2} \left(\frac{2 \times |S_i \cap G_i|}{|S_i|+|G_i|}\right)$$

Thereby, two regions were considered: cells and background. For each region, the dice score was calculated individually
and then weighted equally. With that every region has the same influence on the result in spite of the size.
The background was defined as the largest region and received another value than every other region in order to create a
binary array.
image after clipping to binary array


equation for standard deviation
$$ \sigma = \sqrt{\left(\frac{1}{9}\sum\limits_{i=1}^{9}(x_i-\overline{x}\right)^2}$$
<br>
$$x_i$$
$$\overline{x}$$

equation similarity
$$s=1-\frac{\sigma}{\sigma_{max}}$$
<br>
$$\sigma$$
$$\sigma_{max}$$


equation euclidean distance
$$d_{max} = max_{1\leq i\leq 8} {d_i = \frac{x-x_i}{x}}$$
<br>
$$d_{max}$$
$$d_i$$
$$x$$
$$x_i$$